# <center>Class 5<br>Callbacks Part 2</center>

## Objectives
In this class we will learn:
<ul>
    <li>How to add responsiveness to a dash app</li>
    <li>Learn to use more advanced callbacks</li>
    <li>Some of the problems with dash Input/Output system</li>
</ul>

*These are the main imports needed in any dash web app*<br>
Let's start with the basic imports, any user-defined modules, and any data.

In [ ]:
import dash
from dash import html, dcc
import pandas as pd
from dash import Input, Output, State
import plotly.graph_objects as go


# get the data -> the data comes from the census population growth, there is a python companion program
meta = pd.read_csv('data/metadata.csv')
data = pd.read_csv('data/data.csv')

print(data.head())
print(meta.head())

## Census Population App:
This app will be able to generate multiple graphic types displaying information about US. population estimates.<br>
The data has been downloaded from the us census page. <br>
This is a very dynamic app used to process and show census data over the last decade. The variable names have been coded up in a multi-layer naming structure. This structure will be passed to 4 levels of dropdowns so that the user can choose what to plot and how to plot it.

In [ ]:
# I want to create a function that returns a go Figure which can later be used to add traces to it
def get_figure(fig_title, xtitle = None):
    fig = go.Figure()
    
    fig.update_layout(
        # this is a function taking multiple kwargs where complex args have to be passed as dictionaries
        title = {
            'text': fig_title,
            'y': 0.95,
            'x': 0.5,
            'font': {'size': 22}
        },
        paper_bgcolor = 'white',
        plot_bgcolor = 'white',
        autosize = False,
        height = 400,
        xaxis = {
            'title': xtitle,
            'showline': True, 
            'linewidth': 1,
            'linecolor': 'black'
        },
        yaxis = {
            'showline': True, 
            'linewidth': 1,
            'linecolor': 'black'
        }
    )
    
    return(fig)
get_figure(fig_title = "Title", xtitle = "xTtitle")

In [ ]:
# This makes all possible level-based dropdown options from the meta file
# this is limited to the first 20 just to make it usefull

def get_options(lst):
    return([{'label': l, 'value': l} for l in lst if len(l) < 20])
    # return([{'label': l, 'value': l} for l in lst])

def get_levels():
    levels = {}
    for col in meta.columns:
        if col not in ['id', 'level_0']:
            levels[col] = get_options(meta[col].dropna().drop_duplicates().to_list())
    return(levels)

print(get_levels().keys())
print(get_levels()['level_1'])
# print(get_levels()['level_2'])
# print(get_levels()['level_3'])

## 1. A simple app design
Let's first design an app that has:
<ul>
    <li>One dropdown for each level on the naming for the data-series</li>
    <li>A graph</li>
</ul>

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Initialize the Dash app
app = dash.Dash(__name__)

DROPDOWN_OPTIONS = get_levels()

# Define the initial state of the app -> app.layout is what the user sees upon launching the app
app.layout = html.Div(
    [
        html.H3('Sandbox for Dash Apps'),
        html.H4('Part 1: A simple static app design'),
       
        # There are hidden options here
        html.Hr(),
        html.P('Please select the corresponding series to be plotted. The graph will automatically appear once the final available level has been reached.'),
        html.Div(
            
            # Dynamically create all the dropdowns
            [
                dcc.Dropdown(
                    options=opt,
                    placeholder='Select level {}'.format(name.replace('_', ' ')),
                    disabled=False,
                    searchable=True,
                    multi=False,
                    id='{}_dd'.format(name),
                    style={'width': '10em'}
                ) for name, opt in DROPDOWN_OPTIONS.items()
            ], style={'display': 'flex', 'justify-content': 'space-between'}
        
        ),
        html.Hr(),
        dcc.Graph(figure=get_figure('Custom Census Data Plotter'))
    ], style={'border-style': 'solid', 'padding': '1em'}
)

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True, port=8000)


In [ ]:
print(get_levels()['level_1'][:2])
print(get_levels()['level_2'][:2])
print(get_levels()['level_3'][:2])
print(get_levels()['level_4'][:2])
print(get_levels()['level_5'][:2])

## 2. Functionality
We need the dropdowns to continuously update based on the selection made.<br>
After making a selection of the first, the second must become enabled, and then the third, etc.<br>
Once a terminal selection has been reached, a graph is going to be plotted.

In [ ]:
meta.head()

In [ ]:
meta_tmp = meta.copy()
this_level = True
next_level = False
# values = ['SEX AND AGE', None, None, None, None]
values = ['SEX AND AGE', 'Total population', None, None, None]
# values = ['SEX AND AGE', 'Total population', 'Male', None, None]
# values = ['RACE','Total population','One race','Asian','Filipino'] #, 
# values = ['RACE','Total population','One race','Native Hawaiian and Other Pacific Islander','Samoan'] #, 
for level, val in enumerate(values):
    if val:
        meta_tmp = meta_tmp.loc[meta_tmp['level_{}'.format(level + 1)] == val].copy()
    else:
        this_level = not meta_tmp.loc[pd.isna(meta_tmp['level_{}'.format(level + 1)])].empty
        next_level = True if meta_tmp['level_{}'.format(level + 1)].any() else False
        break
    #     break
with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.max_colwidth', 12):
    print("Subset df:")
    print(meta_tmp.head())
    print("\nOptions for the next level: ")
    print(set(meta_tmp['level_{}'.format(level + 1)]))
    print('\nlevel_{}'.format(level + 1))
print("this_level: ", this_level)
print("next_level: ",next_level)

In [ ]:
def check_next(values):
    '''
    here I want to check if the selected level is a terminal selection
    This returns a tupple:
        True if the current level is a terminal option
        True if next level has options
    
    values is a list of [value/None]
    '''
    meta_tmp = meta.copy()
    this_level = True
    next_level = False
    for level, val in enumerate(values):
        if val:
            meta_tmp = meta_tmp.loc[meta_tmp['level_{}'.format(level + 1)] == val].copy()
        else:
            next_level = True if meta_tmp['level_{}'.format(level + 1)].any() else False
            this_level = not meta_tmp.loc[pd.isna(meta_tmp['level_{}'.format(level + 1)])].empty
            break
    
    return this_level,next_level

print(check_next(['SEX AND AGE', 'Total population', None, None, None]))
print(check_next(['SEX AND AGE', 'Total population', 'Male', None, None]))
print(check_next(['RACE','Total population','One race','Asian','Filipino']))

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate

# Initialize the Dash app
app = dash.Dash(__name__, prevent_initial_callbacks=True)

DROPDOWN_OPTIONS = get_levels()

# Define the initial state of the app -> app.layout is what the user sees upon launching the app
app.layout = html.Div(
    [
        html.H3('Sandbox for Dash Apps'),
        html.H4('Part 2: Adding functionality to the dropdowns'),
       
        # There are hidden options here
        html.Hr(),
        html.P('Please select the corresponding series to be plotted. The graph will automatically appear once the final available level has been reached.'),
        html.Div(
            # Dynamically create all the dropdowns
            [
                dcc.Dropdown(
                    options=opt,
                    placeholder='Select level {}'.format(name.replace('_', ' ')),
                    # Only enable the first dropdown
                    disabled=False if name == 'level_1' else True,
                    searchable=True,
                    multi=False,
                    id='{}_dd'.format(name),
                    style={'width': '10em'}
                ) for name, opt in DROPDOWN_OPTIONS.items()
            ], style={'display': 'flex', 'justify-content': 'space-between'}
        ),
        html.Hr(),
        dcc.Graph(figure=get_figure('Custom Census Data Plotter')),
        html.Div(id='alert')
    ], style={'border-style': 'solid', 'padding': '1em'}
)

# Define callback to enable/disable dropdowns based on selection
@app.callback(
    [
        Output('alert', 'children'),
    ] + [Output('{}_dd'.format(name), 'disabled') for name in DROPDOWN_OPTIONS.keys()],
    [Input('{}_dd'.format(name), 'value') for name in DROPDOWN_OPTIONS.keys()]
)
def check_stuff(*args):
    '''
    The *args contain the values of the selected dropdowns.
    The function handles dropdown events, enabling/disabling dropdowns accordingly.
    '''
    
    # Get callback context
    ctx = dash.callback_context
    if not ctx.triggered:
        raise PreventUpdate

    dropdown = ctx.triggered[0]['prop_id'].split('.')[0]

    # Logic for determining the next dropdown state
    terminal, next = check_next(args)
        
    # Define the alert message
    if terminal:
        alert = f'The selection would result in a plot of {args}'
    else:
        alert = 'Please choose further options'
        
    # Logic for enabling/disabling dropdowns
    if dropdown == 'level_1_dd':
        disabled = [False] + [not next] + [True]*3
    elif dropdown == 'level_2_dd':
        disabled = [False]*2 + [not next] + [True]*2
    elif dropdown == 'level_3_dd':
        disabled = [False]*3 + [not next] + [True]*1
    elif dropdown == 'level_4_dd':
        disabled = [False]*4 + [not next]
    elif dropdown == 'level_5_dd':
        disabled = [False]*5
        
    return [alert] + disabled

# Run the Dash server
if __name__ == '__main__':
    app.run_server(debug=True, port=8010)


In [ ]:
# SEX AND AGE, Total population       -> terminal,    more available
# SEX AND AGE, Total population, Male -> terminal, no more available
# RACE, Total population, One race, Asian, Filipino -> terminal, no more available
# Have in mind that we did subset the choices to choices that are shorter then 20 characters

**Is this working properly? What else do we need to handle?**

## 3. Plotting the graph
Now that we have the basic functionality of the dropdowns. Let's get the plots in.<br>
Here we:
- write a function that returns the graph.
- fixes the dropdown clearing

In [ ]:
# let's add one more thing to the check_next function so that we can get the series name based on the level
def check_next(values):
    '''
    here I want to check if the selected level is a terminal selection
    This returns a tupple:
        True if next level has options
        True if the current level is a terminal option
        
    values is a list of [value/None]
    '''
    meta_tmp = meta.copy()
    this_level = True
    next_level = False
    for level, val in enumerate(values):
        if val:
            meta_tmp = meta_tmp.loc[meta_tmp['level_{}'.format(level + 1)] == val].copy()
        else:
            next_level = True if meta_tmp['level_{}'.format(level + 1)].any() else False
            this_level = not meta_tmp.loc[pd.isna(meta_tmp['level_{}'.format(level + 1)])].empty
            break
    
    series_name = None
    # print(meta_tmp.head())
    if this_level and level < len(values):
        series_name = meta_tmp.loc[pd.isna(meta_tmp['level_{}'.format(level + 1)]), 'id']
        if series_name.empty:
            series_name = meta_tmp.iloc[:,0]            
    else:
        # print("ffffff11")
        series_name = meta_tmp['id']
    
    return this_level, next_level, series_name.squeeze()

print(check_next(['SEX AND AGE', 'Total population', None, None, None]))
# print(check_next(['SEX AND AGE', 'Total population', 'Male', None, None]))
# print(check_next(['RACE','Total population','One race','Asian','Filipino']))

In [ ]:
# data

In [ ]:
def plot_curve(series_name, description, fig):
    this_data = data.loc[data['name'] == series_name].copy()     #in case we need to modify the slice
    x = this_data['year'].to_list()
    y = this_data['data'].to_list()
    
    fig.add_trace(go.Scatter(y = y, x = x, name = description))
    
    return(fig)


# For testing
series_name = 'DP05_0001E'
# series_name = 'DP05_0009E'
description = 'SEX AND AGE-Total population'
fig = get_figure('Testing figure')
fig = plot_curve(series_name, description, fig)
fig.show()

### back to the app development

In [ ]:
series_name = 'DP05_0034E'
# series_name = 'DP05_0009E'
description = 'SEX AND AGE-Total population'
fig = get_figure('Testing figure')
fig = plot_curve(series_name, description, fig)
series_name = 'DP05_0009E'
fig = plot_curve(series_name, description, fig)
fig.show()

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate

app = dash.Dash(__name__)

DROPDOWN_OPTIONS = get_levels()  # Assuming this function is defined elsewhere

# Define the initial state of the app -> app.layout is what the user sees upon launching the app
app.layout = html.Div(
    [
        html.H3('Sandbox for Dash Apps'),
        html.H4('Part 3: Getting a graph'),
       
        # There are hidden options here
        html.Hr(),
        html.P('Please select the corresponding series to be plotted. The graph will automatically appear once the final available level has been reached.'),
        
        # Dynamically create dropdowns
        html.Div(
            [
                dcc.Dropdown(
                    options=opt,
                    placeholder='Select level {}'.format(name.replace('_', ' ')),
                    disabled=False if name == 'level_1' else True,
                    searchable=True,
                    multi=False,
                    id='{}_dd'.format(name),
                    style={'width': '10em'}
                ) for name, opt in DROPDOWN_OPTIONS.items()
            ], style={'display': 'flex', 'justify-content': 'space-between'}
        ),
        
        html.Hr(),
        # Placeholder for the graph
        dcc.Graph(figure=get_figure('Custom Census Data Plotter'), id='figure'),
        # Placeholder for the alert message
        html.Div(id='alert')
    ], style={'border-style': 'solid', 'padding': '1em'}
)

# Callback to update the app based on user inputs
@app.callback(
    [
        Output('alert', 'children'),
    ] + [Output('{}_dd'.format(name), 'disabled') for name in DROPDOWN_OPTIONS.keys()] + [
        Output('figure', 'figure'),
    ],
    [Input('{}_dd'.format(name), 'value') for name in DROPDOWN_OPTIONS.keys()],
)
def check_stuff(*args):
    '''
    The *args contain each one of the ids listed in DROPDOWN_OPTIONS.keys()
    this can be passed into a dictionary to keep track of the selected options.
    
    First, we need to figure out which dropdown was pressed. This works like this:
       1. Capture the context of the triggers
       2. If no context, prevent update
       3. Add the logic to each event
    '''
    
    # Get the context of the triggered input
    ctx = dash.callback_context
    if not ctx.triggered:
        raise PreventUpdate
    
    # Identify the dropdown that triggered the callback
    dropdown = ctx.triggered[0]['prop_id'].split('.')[0]
    
    # Check which dropdown should be enabled next
    terminal, next, series_name = check_next(args)  # Assuming this function is defined elsewhere
        
    # Handle the message displayed in the alert
    if terminal:
        alert = f'The selection would result in a plot of {args}'
    else:
        alert = 'Please choose further options'
    
    # Make None everything after the first None value
    for pos_none, arg in enumerate(args):
        if not arg:
            args = [arg if pos < pos_none else None for pos, arg in enumerate(args)]
    
    # Define the disabled states for each dropdown based on the current selection
    if dropdown == 'level_1_dd':
        disabled = [False] + [not next] + [True]*3
    elif dropdown == 'level_2_dd':
        disabled = [False]*2 + [not next] + [True]*2
    elif dropdown == 'level_3_dd':
        disabled = [False]*3 + [not next] + [True]*1
    elif dropdown == 'level_4_dd':
        disabled = [False]*4 + [not next]
    elif dropdown == 'level_5_dd':
        disabled = [False]*5
    
    # Generate the figure based on the series name
    # print(series_name, description)
    if isinstance(series_name, pd.Series):
        fig = get_figure('Testing figure')
        for l in list(series_name):
            fig = plot_curve(l, description, fig)
    elif isinstance(series_name, str):
        fig = plot_curve(series_name, description, fig=get_figure('A brand new figure'))
    else:
        fig = get_figure('A brand new figure')

    return_list = [alert] + disabled + [fig]
    # print(return_list)
    return return_list

if __name__ == '__main__':
    app.run_server(debug=True, port=8020)


In [ ]:
# SEX AND AGE, Total population       -> terminal,    more available
# SEX AND AGE, Total population, Male -> terminal, no more available
# RACE, Total population, One race, Asian, Filipino -> terminal, no more available

1. The figure is re-created everytime which can be more resource consuming,
2. Recall that plotly figures are objects that have sort of memory, 
3. The State() function allows us to take the current state of any component, or figure object, and use it as needed, 
4. *BUT* we still have a problem, we should have a control to make sure the series is not already added, and
5. We can also add the dropdown values to the outputs so that we can fix the discrepancy after clearing
6. In the same way we can control the figure type
7. In the same way, we can add sub plots